In [2]:
import torch
import numpy as np
import nmmo
from nmmo import config
from neurips2022nmmo import CompetitionConfig, TeamBasedEnv
from nmmo.io import action

class TrainConfig(CompetitionConfig):
    MAP_N = 2

env = TeamBasedEnv(TrainConfig())

obs = env.reset()

obs,reward,done,info = env.step({})

 50%|█████     | 1/2 [00:00<00:00,  9.85it/s]

Generating 2 maps


100%|██████████| 2/2 [00:00<00:00,  9.49it/s]


In [3]:
import yaml
from easydict import EasyDict

with open("./scripts/config/config.yml", 'r') as f:
    arg_file = yaml.full_load(f)
args = EasyDict(arg_file)

In [4]:
for _ in range(5):
    obs,reward,done,info = env.step({})

from utils.envParser import ObservationParser

In [5]:
i=7
myteam=args.self_group_id

parser = ObservationParser()
entities, available, order_in_obs, group_id, global_id, cnt, tiles, move_possible  = parser.parse(observations=obs[myteam], args=args)
entities

array([[0.03808594, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.03222656, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.03417969, 0.015625  , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [6]:
cnt

9

In [7]:
move_possible[:cnt,:]

array([[1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 1.]])

In [8]:
print(global_id[:cnt])
[obs[15][7]['Tile']['Continuous'][j][1] for j in [97,127,113,111,112] ]

[120 123 122 121 124 125 126 -31 127]


[2.0, 2.0, 2.0, 0.0, 2.0]

In [9]:
# obs[15][7]['Tile']['Continuous'][97,2] = 100
# obs[15][7]['Tile']['Continuous'][127,2] = 200
# obs[15][7]['Tile']['Continuous'][113,2] = 300
# obs[15][7]['Tile']['Continuous'][111,2] = 400
# obs[15][7]['Tile']['Continuous'][112,2] = 500
torch.tensor(obs[15][7]['Tile']['Continuous'][:,1]).view(15,15)[5:10,5:10]

tensor([[ 0.,  0.,  2.,  2.,  2.],
        [ 0.,  0.,  2.,  2.,  2.],
        [ 0.,  0.,  2.,  2.,  2.],
        [ 0.,  0.,  2., 13.,  2.],
        [ 0.,  0.,  2.,  2.,  2.]])

In [10]:
torch.tensor(obs[15][7]['Tile']['Continuous'][:,0]).view(15,15)[5:10,5:10]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [11]:
torch.tensor(obs[15][0]['Tile']['Continuous'][:,2]).view(15,15)[5:10,5:10]

tensor([[37., 37., 37., 37., 37.],
        [38., 38., 38., 38., 38.],
        [39., 39., 39., 39., 39.],
        [40., 40., 40., 40., 40.],
        [41., 41., 41., 41., 41.]])

In [12]:
group_id[:cnt].reshape(1,-1)

array([[15, 15, 15, 15, 15, 15, 15, 16, 15]])

In [13]:
global_id[:cnt].reshape(1,-1)

array([[120, 123, 122, 121, 124, 125, 126, -31, 127]])

In [14]:
available[:cnt,i].reshape(1,-1)

array([[0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [15]:
order_in_obs[:cnt,i].reshape(1,-1)

array([[0, 0, 0, 0, 4, 3, 2, 1, 0]])

In [16]:
obs[0][i]['Entity']['Continuous'][:int(obs[myteam][i]['Entity']['N'])]

array([[  1.,   8.,   0.,   1.,   0.,   0.,   0.,  16.,  38.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   5.,   0.,   1.,   0.,   0.,   0.,  16.,  32.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   6.,   0.,   1.,   0.,   0.,   0.,  16.,  34.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1.,   7.,   0.,   1.,   0.,   0.,   0.,  16.,  36.,   0.,   6.,
          0.,   1., 100.,  70.,  70.,   1.,   1.,   1.,   1.,   1.,   1.,
          1.,   1.],
       [  1., -57.,   0.,   1.,   2.,   0.,  -1.,  16.,  44.,   0.,   4.,
          0.,   1., 100., 100., 100.,   1.,   1.,   1.,   0.,   0.,   0.,
          0.,   0.]], dtype=float32)

In [17]:
from model.EntityEncoder import EntityEncoder
e_encoder = EntityEncoder(16)

In [18]:
from model.SpatialEncoder import SpatialEncoder
s_encoder = SpatialEncoder(args = args)

In [19]:
s = torch.Tensor(entities).unsqueeze(dim=0)
n = torch.tensor([cnt])
t = [torch.Tensor(tiles)]
g = torch.tensor(group_id).unsqueeze(dim=0)

sp_embed = s_encoder.forward(t, g)
sp_embed[0,:cnt].sum(axis=-1)

tensor([117.5833, 114.4506, 119.0861, 119.2927, 112.5314, 106.9626, 109.6125,
          0.0000, 109.0163], grad_fn=<SumBackward1>)

In [20]:
sp_embed.shape

torch.Size([1, 512, 256])

In [21]:
en_embed = e_encoder.forward(s,g,sp_embed,n)
en_embed[0,:cnt].sum(-1)

tensor([46.9977, 44.9285, 47.4315, 45.5060, 42.8457, 43.9678, 45.6084, 42.0405,
        46.8413], grad_fn=<SumBackward1>)

In [22]:
en_embed.shape

torch.Size([1, 512, 256])

In [24]:
avg = en_embed.sum(axis = 1)
avg = torch.cat([avg,avg,avg], axis = 0)
n = torch.tensor([cnt])
n = torch.cat([n,n,n/10], axis = 0).reshape(-1,1)
stat = avg/n
stat[:,-1]

tensor([0.7892, 0.7892, 7.8923], grad_fn=<SelectBackward0>)

In [23]:
from model.Core import Core
core = Core()
hidd = core.init_hidden_state(batch_size=3)

hidd[0].shape

torch.Size([1, 3, 128])

In [24]:
ret, hidd = core(stat, hidd)
ret.shape

torch.Size([3, 256])

In [25]:
hidd[0].shape

torch.Size([1, 3, 128])

In [26]:
from model.MoveHead import MoveHead
moveHead = MoveHead()

In [27]:
sp = torch.cat([sp_embed,sp_embed,sp_embed], axis = 0)
mv = torch.tensor([move_possible,move_possible,move_possible], dtype=bool)
act, problog = moveHead(ret, sp, mv, mode='train')

/tmp/ipykernel_4106788/2605343519.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  mv = torch.tensor([move_possible,move_possible,move_possible], dtype=bool)


In [28]:
act.shape

torch.Size([3, 512])

In [29]:
act[0,:cnt]

tensor([1, 0, 2, 1, 2, 1, 0, 2, 0])

In [30]:
problog.shape

torch.Size([3, 512, 5])

In [31]:
problog[0,:cnt]

tensor([[-1.3697, -1.3421, -1.4266,    -inf, -1.4090],
        [-1.3353, -1.3244, -1.4609,    -inf, -1.4316],
        [-1.3404, -1.3522, -1.4615,    -inf, -1.3955],
        [-1.3844, -1.3209, -1.4172,    -inf, -1.4261],
        [-1.3513, -1.3712, -1.4616,    -inf, -1.3648],
        [-1.3328, -1.3546, -1.4489,    -inf, -1.4130],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.3403, -1.3803, -1.4441,    -inf, -1.3833],
        [-1.3513, -1.3566, -1.4374,    -inf, -1.4024]],
       grad_fn=<SliceBackward0>)

In [32]:
gid = torch.tensor([group_id,group_id,group_id],dtype=int)
gid[:,:cnt]

tensor([[15, 15, 15, 15, 15, 15, 16, 15, 15],
        [15, 15, 15, 15, 15, 15, 16, 15, 15],
        [15, 15, 15, 15, 15, 15, 16, 15, 15]])

In [33]:
i=1
idx = (gid[i]==args.self_group_id).nonzero().view(-1)
gid[i,idx]

tensor([15, 15, 15, 15, 15, 15, 15, 15])

In [34]:
id = global_id[idx]%8
id

array([0, 3, 2, 1, 4, 5, 6, 7])

In [35]:
act[i,idx]

tensor([1, 2, 4, 1, 2, 1, 1, 4])

In [36]:
problog[i,idx]

tensor([[-1.3697, -1.3421, -1.4266,    -inf, -1.4090],
        [-1.3353, -1.3244, -1.4609,    -inf, -1.4316],
        [-1.3404, -1.3522, -1.4615,    -inf, -1.3955],
        [-1.3844, -1.3209, -1.4172,    -inf, -1.4261],
        [-1.3513, -1.3712, -1.4616,    -inf, -1.3648],
        [-1.3328, -1.3546, -1.4489,    -inf, -1.4130],
        [-1.3403, -1.3803, -1.4441,    -inf, -1.3833],
        [-1.3513, -1.3566, -1.4374,    -inf, -1.4024]],
       grad_fn=<IndexBackward0>)

In [37]:
from model.AttackHead import AttackHead
atkHead = AttackHead()

In [43]:
ret.shape
# act, problog = moveHead(ret, sp, mv, mode='train')

torch.Size([3, 8, 512])

In [44]:
en = torch.cat([en_embed,en_embed,en_embed], axis = 0)
en.shape

torch.Size([3, 512, 256])

In [45]:
av = torch.tensor([available,available,available], dtype=bool).transpose(1,2)
av.shape

torch.Size([3, 8, 512])

In [59]:
def get_self_embedd(en_embed, global_id, group_id):
    idx = (group_id==args.self_group_id).nonzero().view(-1)
    real_id = global_id[idx]%8
    np.zeros(())
    print(real_id)
get_self_embedd(en_embed, global_id, gid[0])

[0 3 2 1 4 5 6 7]
